In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns
import os

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [4]:
D = 30
N = 10000
z_size = 8

# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.0001
b1 = 0.9
b2 = 0.999

global_t = 4
k_lab = [D//10, D//6, D//3, D//2, D]
trial_num = 1

In [5]:
train_data, test_data = generate_synthetic_data_with_noise(N, z_size, D)

In [6]:
def train_model(train_data, model):
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=lr, 
                                 betas = (b1,b2))
        
    for epoch in range(1, n_epochs+1):
        train(train_data, 
              model, 
              optimizer, 
              epoch, 
              batch_size)
        model.t = max(0.001, model.t * 0.99)

        
    return model

In [7]:
def save_model(base_path, model):
    # make directory
    if not os.path.exists(os.path.dirname(base_path)):
        try:
            os.makedirs(os.path.dirname(base_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise Exception("COULD NOT MAKE PATH")
    with open(base_path, 'wb') as PATH:
        torch.save(model.state_dict(), PATH)

In [8]:
BASE_PATH = "../data/models/final_run/"
#BASE_PATH = '/scratch/ns3429/sparse-subset/data/models/final_run/'

Vanilla VAE Gumbel First

In [9]:
def run_vanilla_vae_gumbel():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("VANILLA VAE GUMBEL, TRIAL={}, K={}".format(trial, k))
            additional_path = "vanilla_vae_gumbel/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [10]:
#run_vanilla_vae_gumbel()

Batching Gumbel VAE

In [11]:
def run_batching_gumbel_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("BATCHING GUMBEL VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "batching_gumbel_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_NInsta(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [12]:
#run_batching_gumbel_vae()

Global Gate VAE

In [13]:
def run_globalgate_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("GLOBAL GATE VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "globalgate_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_GlobalGate(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [14]:
#run_globalgate_vae()

RunningState VAE

In [15]:
def run_runningstate_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("RUNNING STATE VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "runningstate_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_RunningState(2*D, 100, 20, k = k, t = global_t, alpha = 0.9)
            model.to(device)
            train_model(train_data, model)
            model.set_burned_in()
            save_model(BASE_PATH + additional_path, model)

In [16]:
#run_runningstate_vae()

ConcreteVAE

In [17]:
def run_concrete_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("CONCRETE VAE NMSL, TRIAL={}, K={}".format(trial, k))
            additional_path = "concrete_vae_nmsl/k_{}/model_trial_{}.pt".format(k, trial)
            model = ConcreteVAE_NMSL(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [18]:
#run_concrete_vae()

In [19]:
model = ConcreteVAE_NMSL(2*D, 100, 20, k = 15, t = global_t)
model.to(device)

ConcreteVAE_NMSL(
  (encoder): Sequential(
    (0): Linear(in_features=15, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=100, out_features=20, bias=True)
  (enc_logvar): Linear(in_features=100, out_features=20, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=60, bias=True)
    (3): Sigmoid()
  )
)

In [20]:
model.encoder[0].weight

Parameter containing:
tensor([[-0.2120, -0.2559, -0.0879,  ...,  0.0740,  0.2447,  0.1658],
        [ 0.0872,  0.0551, -0.0839,  ...,  0.0939,  0.1282,  0.1712],
        [-0.1744, -0.0149, -0.0548,  ...,  0.1778, -0.2318, -0.0114],
        ...,
        [ 0.0098,  0.0148,  0.1773,  ...,  0.0473,  0.1313, -0.2416],
        [ 0.2328,  0.0861,  0.0975,  ...,  0.0768,  0.0668, -0.2169],
        [ 0.2397,  0.0831, -0.0517,  ...,  0.1488, -0.0291, -0.2208]],
       device='cuda:0', requires_grad=True)

In [21]:
model.load_state_dict(torch.load(BASE_PATH+"concrete_vae_nmsl/k_15/model_trial_1.pt"))
model.eval()

ConcreteVAE_NMSL(
  (encoder): Sequential(
    (0): Linear(in_features=15, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=100, out_features=20, bias=True)
  (enc_logvar): Linear(in_features=100, out_features=20, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=60, bias=True)
    (3): Sigmoid()
  )
)

In [22]:
print("Ran successfully")

Ran successfully


In [23]:
run_runningstate_vae()

RUNNING STATE VAE, TRIAL=1, K=3
Train Epoch: 1 [0/8000 (0%)]	Loss: 42.251862
Train Epoch: 1 [1280/8000 (16%)]	Loss: 41.723572
Train Epoch: 1 [2560/8000 (32%)]	Loss: 40.980740
Train Epoch: 1 [3840/8000 (48%)]	Loss: 40.587543
Train Epoch: 1 [5120/8000 (64%)]	Loss: 40.121208
Train Epoch: 1 [6400/8000 (80%)]	Loss: 39.457130
Train Epoch: 1 [7680/8000 (96%)]	Loss: 39.212852
====> Epoch: 1 Average loss: 40.5790
Train Epoch: 2 [0/8000 (0%)]	Loss: 38.972275
Train Epoch: 2 [1280/8000 (16%)]	Loss: 38.512249
Train Epoch: 2 [2560/8000 (32%)]	Loss: 38.026398
Train Epoch: 2 [3840/8000 (48%)]	Loss: 37.466541
Train Epoch: 2 [5120/8000 (64%)]	Loss: 37.004318
Train Epoch: 2 [6400/8000 (80%)]	Loss: 36.584988
Train Epoch: 2 [7680/8000 (96%)]	Loss: 35.974052
====> Epoch: 2 Average loss: 37.5131
Train Epoch: 3 [0/8000 (0%)]	Loss: 35.989693
Train Epoch: 3 [1280/8000 (16%)]	Loss: 35.787285
Train Epoch: 3 [2560/8000 (32%)]	Loss: 34.911983
Train Epoch: 3 [3840/8000 (48%)]	Loss: 34.655479
Train Epoch: 3 [5120/800

Train Epoch: 22 [6400/8000 (80%)]	Loss: 32.188461
Train Epoch: 22 [7680/8000 (96%)]	Loss: 32.136105
====> Epoch: 22 Average loss: 32.1366
Train Epoch: 23 [0/8000 (0%)]	Loss: 32.248714
Train Epoch: 23 [1280/8000 (16%)]	Loss: 31.904510
Train Epoch: 23 [2560/8000 (32%)]	Loss: 32.422363
Train Epoch: 23 [3840/8000 (48%)]	Loss: 32.019646
Train Epoch: 23 [5120/8000 (64%)]	Loss: 31.915089
Train Epoch: 23 [6400/8000 (80%)]	Loss: 32.227905
Train Epoch: 23 [7680/8000 (96%)]	Loss: 31.967524
====> Epoch: 23 Average loss: 32.0706
Train Epoch: 24 [0/8000 (0%)]	Loss: 31.780191
Train Epoch: 24 [1280/8000 (16%)]	Loss: 31.528877
Train Epoch: 24 [2560/8000 (32%)]	Loss: 32.618031
Train Epoch: 24 [3840/8000 (48%)]	Loss: 32.092743
Train Epoch: 24 [5120/8000 (64%)]	Loss: 31.945189
Train Epoch: 24 [6400/8000 (80%)]	Loss: 32.083672
Train Epoch: 24 [7680/8000 (96%)]	Loss: 32.226917
====> Epoch: 24 Average loss: 32.0135
Train Epoch: 25 [0/8000 (0%)]	Loss: 31.954718
Train Epoch: 25 [1280/8000 (16%)]	Loss: 32.09153

Train Epoch: 44 [1280/8000 (16%)]	Loss: 32.335026
Train Epoch: 44 [2560/8000 (32%)]	Loss: 31.515526
Train Epoch: 44 [3840/8000 (48%)]	Loss: 31.414705
Train Epoch: 44 [5120/8000 (64%)]	Loss: 31.880751
Train Epoch: 44 [6400/8000 (80%)]	Loss: 31.770123
Train Epoch: 44 [7680/8000 (96%)]	Loss: 32.019596
====> Epoch: 44 Average loss: 31.8299
Train Epoch: 45 [0/8000 (0%)]	Loss: 31.823887
Train Epoch: 45 [1280/8000 (16%)]	Loss: 31.808435
Train Epoch: 45 [2560/8000 (32%)]	Loss: 32.074623
Train Epoch: 45 [3840/8000 (48%)]	Loss: 31.843147
Train Epoch: 45 [5120/8000 (64%)]	Loss: 32.005692
Train Epoch: 45 [6400/8000 (80%)]	Loss: 31.953489
Train Epoch: 45 [7680/8000 (96%)]	Loss: 31.742865
====> Epoch: 45 Average loss: 31.8248
Train Epoch: 46 [0/8000 (0%)]	Loss: 31.805288
Train Epoch: 46 [1280/8000 (16%)]	Loss: 31.987476
Train Epoch: 46 [2560/8000 (32%)]	Loss: 31.694292
Train Epoch: 46 [3840/8000 (48%)]	Loss: 31.615719
Train Epoch: 46 [5120/8000 (64%)]	Loss: 31.793728
Train Epoch: 46 [6400/8000 (80%)

Train Epoch: 65 [6400/8000 (80%)]	Loss: 31.670469
Train Epoch: 65 [7680/8000 (96%)]	Loss: 31.409817
====> Epoch: 65 Average loss: 31.6714
Train Epoch: 66 [0/8000 (0%)]	Loss: 31.649094
Train Epoch: 66 [1280/8000 (16%)]	Loss: 31.354994
Train Epoch: 66 [2560/8000 (32%)]	Loss: 31.328386
Train Epoch: 66 [3840/8000 (48%)]	Loss: 32.024387
Train Epoch: 66 [5120/8000 (64%)]	Loss: 31.490622
Train Epoch: 66 [6400/8000 (80%)]	Loss: 31.421053
Train Epoch: 66 [7680/8000 (96%)]	Loss: 31.022448
====> Epoch: 66 Average loss: 31.6664
Train Epoch: 67 [0/8000 (0%)]	Loss: 31.440578
Train Epoch: 67 [1280/8000 (16%)]	Loss: 31.348391
Train Epoch: 67 [2560/8000 (32%)]	Loss: 31.690090
Train Epoch: 67 [3840/8000 (48%)]	Loss: 31.632759
Train Epoch: 67 [5120/8000 (64%)]	Loss: 31.284658
Train Epoch: 67 [6400/8000 (80%)]	Loss: 31.632715
Train Epoch: 67 [7680/8000 (96%)]	Loss: 31.212917
====> Epoch: 67 Average loss: 31.6459
Train Epoch: 68 [0/8000 (0%)]	Loss: 31.664291
Train Epoch: 68 [1280/8000 (16%)]	Loss: 31.52070

Train Epoch: 87 [1280/8000 (16%)]	Loss: 31.803148
Train Epoch: 87 [2560/8000 (32%)]	Loss: 31.375805
Train Epoch: 87 [3840/8000 (48%)]	Loss: 31.510395
Train Epoch: 87 [5120/8000 (64%)]	Loss: 31.733740
Train Epoch: 87 [6400/8000 (80%)]	Loss: 31.587959
Train Epoch: 87 [7680/8000 (96%)]	Loss: 31.492079
====> Epoch: 87 Average loss: 31.5572
Train Epoch: 88 [0/8000 (0%)]	Loss: 31.408953
Train Epoch: 88 [1280/8000 (16%)]	Loss: 31.809528
Train Epoch: 88 [2560/8000 (32%)]	Loss: 32.123802
Train Epoch: 88 [3840/8000 (48%)]	Loss: 31.335018
Train Epoch: 88 [5120/8000 (64%)]	Loss: 31.318327
Train Epoch: 88 [6400/8000 (80%)]	Loss: 31.704359
Train Epoch: 88 [7680/8000 (96%)]	Loss: 31.448854
====> Epoch: 88 Average loss: 31.5484
Train Epoch: 89 [0/8000 (0%)]	Loss: 31.611084
Train Epoch: 89 [1280/8000 (16%)]	Loss: 31.982939
Train Epoch: 89 [2560/8000 (32%)]	Loss: 31.679863
Train Epoch: 89 [3840/8000 (48%)]	Loss: 31.475649
Train Epoch: 89 [5120/8000 (64%)]	Loss: 31.760778
Train Epoch: 89 [6400/8000 (80%)

...

Train Epoch: 511 [6400/8000 (80%)]	Loss: 29.980915
Train Epoch: 511 [7680/8000 (96%)]	Loss: 30.055029
====> Epoch: 511 Average loss: 29.8901
Train Epoch: 512 [0/8000 (0%)]	Loss: 29.907923
Train Epoch: 512 [1280/8000 (16%)]	Loss: 29.695862
Train Epoch: 512 [2560/8000 (32%)]	Loss: 30.010572
Train Epoch: 512 [3840/8000 (48%)]	Loss: 29.847425
Train Epoch: 512 [5120/8000 (64%)]	Loss: 29.934771
Train Epoch: 512 [6400/8000 (80%)]	Loss: 29.662193
Train Epoch: 512 [7680/8000 (96%)]	Loss: 29.537449
====> Epoch: 512 Average loss: 29.9014
Train Epoch: 513 [0/8000 (0%)]	Loss: 30.184393
Train Epoch: 513 [1280/8000 (16%)]	Loss: 30.032518
Train Epoch: 513 [2560/8000 (32%)]	Loss: 29.614042
Train Epoch: 513 [3840/8000 (48%)]	Loss: 29.839758
Train Epoch: 513 [5120/8000 (64%)]	Loss: 29.592005
Train Epoch: 513 [6400/8000 (80%)]	Loss: 29.669516
Train Epoch: 513 [7680/8000 (96%)]	Loss: 30.227726
====> Epoch: 513 Average loss: 29.8936
Train Epoch: 514 [0/8000 (0%)]	Loss: 29.839119
Train Epoch: 514 [1280/8000 

Train Epoch: 532 [6400/8000 (80%)]	Loss: 29.815531
Train Epoch: 532 [7680/8000 (96%)]	Loss: 29.792599
====> Epoch: 532 Average loss: 29.8740
Train Epoch: 533 [0/8000 (0%)]	Loss: 29.944942
Train Epoch: 533 [1280/8000 (16%)]	Loss: 29.589470
Train Epoch: 533 [2560/8000 (32%)]	Loss: 29.936035
Train Epoch: 533 [3840/8000 (48%)]	Loss: 29.619305
Train Epoch: 533 [5120/8000 (64%)]	Loss: 29.875998
Train Epoch: 533 [6400/8000 (80%)]	Loss: 29.975754
Train Epoch: 533 [7680/8000 (96%)]	Loss: 29.972717
====> Epoch: 533 Average loss: 29.8725
Train Epoch: 534 [0/8000 (0%)]	Loss: 29.655327
Train Epoch: 534 [1280/8000 (16%)]	Loss: 30.003122
Train Epoch: 534 [2560/8000 (32%)]	Loss: 29.705629
Train Epoch: 534 [3840/8000 (48%)]	Loss: 29.629456
Train Epoch: 534 [5120/8000 (64%)]	Loss: 29.907560
Train Epoch: 534 [6400/8000 (80%)]	Loss: 30.080788
Train Epoch: 534 [7680/8000 (96%)]	Loss: 29.784750
====> Epoch: 534 Average loss: 29.8634
Train Epoch: 535 [0/8000 (0%)]	Loss: 29.615879
Train Epoch: 535 [1280/8000 

Train Epoch: 553 [6400/8000 (80%)]	Loss: 29.645023
Train Epoch: 553 [7680/8000 (96%)]	Loss: 29.951536
====> Epoch: 553 Average loss: 29.8511
Train Epoch: 554 [0/8000 (0%)]	Loss: 29.674616
Train Epoch: 554 [1280/8000 (16%)]	Loss: 29.887022
Train Epoch: 554 [2560/8000 (32%)]	Loss: 29.995398
Train Epoch: 554 [3840/8000 (48%)]	Loss: 30.192583
Train Epoch: 554 [5120/8000 (64%)]	Loss: 29.768904
Train Epoch: 554 [6400/8000 (80%)]	Loss: 30.106407
Train Epoch: 554 [7680/8000 (96%)]	Loss: 30.128645
====> Epoch: 554 Average loss: 29.8409
Train Epoch: 555 [0/8000 (0%)]	Loss: 30.087805
Train Epoch: 555 [1280/8000 (16%)]	Loss: 29.888390
Train Epoch: 555 [2560/8000 (32%)]	Loss: 30.100567
Train Epoch: 555 [3840/8000 (48%)]	Loss: 29.937370
Train Epoch: 555 [5120/8000 (64%)]	Loss: 29.999969
Train Epoch: 555 [6400/8000 (80%)]	Loss: 29.753794
Train Epoch: 555 [7680/8000 (96%)]	Loss: 29.853840
====> Epoch: 555 Average loss: 29.8434
Train Epoch: 556 [0/8000 (0%)]	Loss: 29.798311
Train Epoch: 556 [1280/8000 

Train Epoch: 574 [6400/8000 (80%)]	Loss: 29.727598
Train Epoch: 574 [7680/8000 (96%)]	Loss: 29.992764
====> Epoch: 574 Average loss: 29.8230
Train Epoch: 575 [0/8000 (0%)]	Loss: 29.602884
Train Epoch: 575 [1280/8000 (16%)]	Loss: 29.642090
Train Epoch: 575 [2560/8000 (32%)]	Loss: 29.832270
Train Epoch: 575 [3840/8000 (48%)]	Loss: 29.897003
Train Epoch: 575 [5120/8000 (64%)]	Loss: 29.929113
Train Epoch: 575 [6400/8000 (80%)]	Loss: 29.668182
Train Epoch: 575 [7680/8000 (96%)]	Loss: 29.685905
====> Epoch: 575 Average loss: 29.8347
Train Epoch: 576 [0/8000 (0%)]	Loss: 29.859318
Train Epoch: 576 [1280/8000 (16%)]	Loss: 29.968077
Train Epoch: 576 [2560/8000 (32%)]	Loss: 29.961397
Train Epoch: 576 [3840/8000 (48%)]	Loss: 29.630255
Train Epoch: 576 [5120/8000 (64%)]	Loss: 29.870613
Train Epoch: 576 [6400/8000 (80%)]	Loss: 29.857384
Train Epoch: 576 [7680/8000 (96%)]	Loss: 29.996489
====> Epoch: 576 Average loss: 29.8258
Train Epoch: 577 [0/8000 (0%)]	Loss: 29.911163
Train Epoch: 577 [1280/8000 

Train Epoch: 595 [6400/8000 (80%)]	Loss: 29.532303
Train Epoch: 595 [7680/8000 (96%)]	Loss: 30.079752
====> Epoch: 595 Average loss: 29.7977
Train Epoch: 596 [0/8000 (0%)]	Loss: 29.765375
Train Epoch: 596 [1280/8000 (16%)]	Loss: 29.757301
Train Epoch: 596 [2560/8000 (32%)]	Loss: 29.650101
Train Epoch: 596 [3840/8000 (48%)]	Loss: 30.074316
Train Epoch: 596 [5120/8000 (64%)]	Loss: 29.749107
Train Epoch: 596 [6400/8000 (80%)]	Loss: 29.994431
Train Epoch: 596 [7680/8000 (96%)]	Loss: 30.048567
====> Epoch: 596 Average loss: 29.8118
Train Epoch: 597 [0/8000 (0%)]	Loss: 30.001913
Train Epoch: 597 [1280/8000 (16%)]	Loss: 30.081535
Train Epoch: 597 [2560/8000 (32%)]	Loss: 30.071552
Train Epoch: 597 [3840/8000 (48%)]	Loss: 29.720425
Train Epoch: 597 [5120/8000 (64%)]	Loss: 30.016592
Train Epoch: 597 [6400/8000 (80%)]	Loss: 29.854076
Train Epoch: 597 [7680/8000 (96%)]	Loss: 30.024218
====> Epoch: 597 Average loss: 29.8095
Train Epoch: 598 [0/8000 (0%)]	Loss: 29.762173
Train Epoch: 598 [1280/8000 